##### URLs

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>url</th>
      <th>checkbox</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td><a href="https://player.vimeo.com/video/656842813?h=07f074324e">https://player.vimeo.com/video/656842813?h=07f074324e</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>1</th>
      <td><a href="https://player.vimeo.com/video/656842538?h=bd85634eb1">https://player.vimeo.com/video/656842538?h=bd85634eb1</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>2</th>
      <td><a href="https://player.vimeo.com/video/656842076?h=0f654d427f">https://player.vimeo.com/video/656842076?h=0f654d427f</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>3</th>
      <td><a href="https://player.vimeo.com/video/656841910?h=79c7dbc5ab">https://player.vimeo.com/video/656841910?h=79c7dbc5ab</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>4</th>
      <td><a href="https://player.vimeo.com/video/656848648?h=6964fa0c8c">https://player.vimeo.com/video/656848648?h=6964fa0c8c</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>5</th>
      <td><a href="https://player.vimeo.com/video/656848196?h=37dbc44b09">https://player.vimeo.com/video/656848196?h=37dbc44b09</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>6</th>
      <td><a href="https://player.vimeo.com/video/656847896?h=11de775937">https://player.vimeo.com/video/656847896?h=11de775937</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>7</th>
      <td><a href="https://player.vimeo.com/video/656847237?h=4b5cfed5d6">https://player.vimeo.com/video/656847237?h=4b5cfed5d6</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>8</th>
      <td><a href="https://player.vimeo.com/video/656849559?h=bca444c8af">https://player.vimeo.com/video/656849559?h=bca444c8af</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>9</th>
      <td><a href="https://player.vimeo.com/video/656849505?h=f153a4f005">https://player.vimeo.com/video/656849505?h=f153a4f005</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>10</th>
      <td><a href="https://player.vimeo.com/video/656849254?h=e6faad47ca">https://player.vimeo.com/video/656849254?h=e6faad47ca</a></td>
      <td><input type="checkbox"></td>
    </tr>
    <tr>
      <th>11</th>
      <td><a href="https://player.vimeo.com/video/656848911?h=7939dcd479">https://player.vimeo.com/video/656848911?h=7939dcd479</a></td>
      <td><input type="checkbox"></td>
    </tr>
  </tbody>
</table>